# 01 - Stažení dat

## Cíl tohoto notebooku

V tomto kroku stáhneme HTML stránky s hokejovými statistikami z webu [Scrape This Site](https://www.scrapethissite.com/pages/forms/).

**Co se naučíte:**
- Použití Selenium pro automatizaci prohlížeče
- Analýzu struktury URL pro stránkování
- Ukládání HTML souborů na disk

**Výstup:** Složka `data/raw/` naplněná HTML soubory (jeden soubor pro každou stránku dat)

---

## Krok 1: Příprava prostředí

### 1.1 Import potřebných knihoven

In [1]:
# Doplňte importy potřebných knihoven:
# - selenium webdriver pro ovládání prohlížeče
# - time pro pauzy mezi požadavky
# - os pro práci se složkami

from selenium import webdriver
import time
import os

### 1.2 Vytvoření adresářové struktury

Vytvořte složku `data/raw/`, pokud ještě neexistuje.

In [16]:
# Vytvořte složku pro surová data
# Použijte os.makedirs() s parametrem exist_ok=True
CESTA_RAW = '../data/raw'

os.makedirs(CESTA_RAW, exist_ok=True)

---

### 1.3 Analýza webu

Před psaním kódu si prohlédněte web:
1. Otevřete https://www.scrapethissite.com/pages/forms/
2. Prohlédněte si, jak vypadá tabulka s daty
3. Klikněte na stránkování dole a sledujte, jak se mění URL

**Otázky k zamyšlení:**
- Kolik stránek s daty je k dispozici?
- Jak vypadá URL pro druhou stránku? Třetí stránku?
- Jaký je vzor URL pro stránkování?

#### 1.3.1 Definice základní URL

Na základě analýzy webu definujte:
- Základní URL pro stránku s formuláři


In [4]:
# Definujte konstanty
# URL má tvar: https://www.scrapethissite.com/pages/forms/?page_num=X

# Doplňte základní URL - bez čísla na konci

ZAKLADNI_URL = "https://www.scrapethissite.com/pages/forms/?page_num="


---

### 1.4 Konfigurace a spuštění Selenium

Vytvořte instanci Chrome prohlížeče. Novější verze Selenium automaticky stáhnou potřebný driver.

In [5]:
# Spuštění prohlížeče Chrome
browser = webdriver.Chrome()

<details>
<summary>Nápověda</summary>

```python
browser = webdriver.Chrome()

# Pro Operu - Opera pouziva ChromeDriver:
from selenium.webdriver.chrome.options import Options
options = Options()
options.binary_location = 'cesta k souboru .... opera.exe' # tu musíte nastavit cestu k souboru opera.exe
browser = webdriver.Chrome(options=options)
```
</details>

In [6]:
# Otestujte připojení k stránce
browser.get(ZAKLADNI_URL + "1")

In [7]:
browser.title

'Hockey Teams: Forms, Searching and Pagination | Scrape This Site | A public sandbox for learning web scraping'

---

## Krok 2: Funkce `je_tabulka_prazdna()` pro detekci prázdné stránky na konci cyklu

### 2.1 Analýza struktury tabulky

Prozkoumejte HTML strukturu stránky pomocí Developer Tools (F12):
- Jak je označena tabulka s daty?
- Jak jsou označeny řádky s týmy?
- Co se stane, když na stránce nejsou žádná data?

In [11]:
# Prozkoumejte strukturu - najděte řádky tabulky s třídou 'team'
browser.get(ZAKLADNI_URL + "1")
time.sleep(1)

radky = browser.find_elements('css selector', 'tr.team')
len(radky)

25

<details>
<summary>Nápověda</summary>

Použijte find_elements() s parametrem 'class name' nebo 'css selector'

</details>

In [12]:
# Zkuste stránku bez dat - jakoze 100. v poradí, kolik tříd 'team' najde
browser.get(ZAKLADNI_URL + "100")
time.sleep(1)
radky = browser.find_elements('css selector', 'tr.team')
len(radky)

0

### 2.2 Implementace funkce `je_tabulka_prazdna()`

Vytvořte funkci, která zkontroluje, zda aktuálně načtená stránka obsahuje data.

Funkce vrátí:
- `True` - pokud tabulka neobsahuje žádné řádky s daty
- `False` - pokud tabulka obsahuje data

In [13]:
def je_tabulka_prazdna(browser) -> bool:
    radky = browser.find_elements('css selector', 'tr.team')
    return len(radky) == 0

<details>
<summary>Nápověda</summary>
    
    # v browseri najděte všechny elementy s třídou 'team' - cez metodu `find_elements()` s parametrem 'css selector' nebo 'class name'
    # Pokud je jejich počet 0, tabulka je prázdná
</details>

### 2.3 Test funkce

In [14]:
# Otestujte vytvořenou funkci na stránce s daty (1. strana) a bez dat (100. strana)
browser.get(ZAKLADNI_URL + "2")
time.sleep(1)

je_tabulka_prazdna(browser)

False

In [15]:
browser.get(ZAKLADNI_URL + "25")
time.sleep(1)

je_tabulka_prazdna(browser)

True

---

## Krok 3: Funkce `uloz_html` - Stažení HTML stránek

### 3.1 Funkce pro uložení HTML

Nejprve vytvořte pomocnou funkci pro uložení HTML obsahu do souboru.

In [19]:
cislo_stranky = 1
str(cislo_stranky).zfill(2)

'01'

In [22]:
HTML_NAME = f"{CESTA_RAW}/hockey_teams_page_"
HTML_NAME

'../data/raw/hockey_teams_page_'

In [23]:
f"{HTML_NAME}{str(cislo_stranky).zfill(2)}.html"

'../data/raw/hockey_teams_page_01.html'

In [25]:
def uloz_html(html_obsah, cislo_stranky):
    """
    Uloží HTML obsah do souboru.
    
    Parametry:
        html_obsah: HTML kód stránky (string)
        cislo_stranky: Číslo stránky pro název souboru (int)
    
    Název souboru bude ve formátu: hockey_teams_page_01.html
    """
    nazev_souboru = f"{HTML_NAME}{str(cislo_stranky).zfill(2)}.html"
    with open(nazev_souboru, 'w', encoding='utf-8') as f:
        f.write(html_obsah)

<details>
<summary>Nápověda</summary>
1) Ve funkci vytvořte řetězec(string) jako

* cestu do data/raw
* název souboru + číslo na 2 místa + .html

2) Otevřete soubor pro zápis

3) Zapište obsah v proměnné html_obsah
</details>

In [27]:
html_obsah = "<html>neco</html>"
uloz_html(html_obsah, 3)

### 3.2 Hlavní smyčka pro stažení všech stránek

Napište `while` cyklus, který:
1. Načte stránku
2. Zkontroluje, zda tabulka obsahuje data
3. Pokud ano - uloží HTML a přejde na další stránku
4. Pokud ne - ukončí cyklus

**Důležité:** 
- Použijte podmínku `while not je_tabulka_prazdna(browser)`
- Nezapomeňte na pauzu mezi požadavky (respektujte FAQ webu!)
- Stránku je třeba nejprve načíst, než můžete zkontrolovat, zda je prázdná

In [32]:
# Stažení všech stránek
# Nezapomeňte na pauzu mezi požadavky pomocí time.sleep()

# nacteme prvni stranku
str_num = 1
browser.get(ZAKLADNI_URL+str(str_num))
time.sleep(1)

while not je_tabulka_prazdna(browser):
    print('Stahuji stránku: ' + ZAKLADNI_URL+str(str_num))
    # stahni a uloz
    obsah = browser.page_source
    uloz_html(obsah, str_num)

    # prechod na dalsi stranku
    str_num += 1
    browser.get(ZAKLADNI_URL+str(str_num))
    time.sleep(1)

print(f'Stahování dokonceno. Stáhnutých {str_num - 1} stránek.')

Stahuji stránku: https://www.scrapethissite.com/pages/forms/?page_num=1
Stahuji stránku: https://www.scrapethissite.com/pages/forms/?page_num=2
Stahuji stránku: https://www.scrapethissite.com/pages/forms/?page_num=3
Stahuji stránku: https://www.scrapethissite.com/pages/forms/?page_num=4
Stahuji stránku: https://www.scrapethissite.com/pages/forms/?page_num=5
Stahuji stránku: https://www.scrapethissite.com/pages/forms/?page_num=6
Stahuji stránku: https://www.scrapethissite.com/pages/forms/?page_num=7
Stahuji stránku: https://www.scrapethissite.com/pages/forms/?page_num=8
Stahuji stránku: https://www.scrapethissite.com/pages/forms/?page_num=9
Stahuji stránku: https://www.scrapethissite.com/pages/forms/?page_num=10
Stahuji stránku: https://www.scrapethissite.com/pages/forms/?page_num=11
Stahuji stránku: https://www.scrapethissite.com/pages/forms/?page_num=12
Stahuji stránku: https://www.scrapethissite.com/pages/forms/?page_num=13
Stahuji stránku: https://www.scrapethissite.com/pages/forms/

### 3.3 Zavření prohlížeče

Po dokončení stahování nezapomeňte zavřít prohlížeč.

In [33]:
# Zavřete prohlížeč
browser.quit()



---

## Krok 5: Ověření výsledků

### 5.1 Kontrola stažených souborů

Zkontrolujte, kolik souborů bylo staženo a vypište jejich názvy.

In [36]:
CESTA_RAW + '/*.html'

'../data/raw/*.html'

In [44]:
CESTA_RAW

'../data/raw'

In [37]:
import glob

# Najděte všechny HTML soubory ve složce data/raw/ a vypište jejich počet a názvy
subory = glob.glob(CESTA_RAW + '/*.html')
len(subory)

24

In [38]:
subory[-1]

'../data/raw\\hockey_teams_page_24.html'

In [40]:
subory[:3]

['../data/raw\\hockey_teams_page_01.html',
 '../data/raw\\hockey_teams_page_02.html',
 '../data/raw\\hockey_teams_page_03.html']

<details>
<summary>Nápověda</summary>

použi funkci `glob.glob()`, 
seznam seřaď pomocí `sort()`

v cyklu vypište seznam souborů, případne délku seznamu
</details>

### 6.2 Kontrola obsahu prvního souboru

Načtěte první soubor a zobrazte prvních 500 znaků pro ověření, že obsahuje správná data.

In [43]:
# Načtěte a zobrazte začátek prvního HTML souboru

nazev_souboru = f"{HTML_NAME}01.html"

with open(nazev_souboru, 'r', encoding='utf-8') as f:
    test_obsah = f.read()

test_obsah[:500]

'<html lang="en"><head>\n    <meta charset="utf-8">\n    <title>Hockey Teams: Forms, Searching and Pagination | Scrape This Site | A public sandbox for learning web scraping</title>\n    <link rel="icon" type="image/png" href="/static/images/scraper-icon.png">\n\n    <meta name="viewport" content="width=device-width, initial-scale=1.0">\n    <meta name="description" content="Browse through a database of NHL team stats since 1990. Practice building a scraper that handles common website interface compone'

<details>
<summary>Nápověda</summary>

```python
with open('data/raw/hockey_teams_page_01.html', 'r', encoding='utf-8') as f:
    obsah = f.read()

print(obsah[:500])
```
</details>

---

## Shrnutí

V tomto notebooku jste zkusili:
- Používat Selenium pro automatizaci webového prohlížeče
- Analyzovat strukturu URL pro stránkování
- Stahovat a ukládat HTML soubory
- Respektovat limity webu pomocí pauz mezi požadavky

**Výstup:** Ve složce `data/raw/` máte nyní 24 HTML souborů s hokejovými statistikami.

**Další krok:** Přejděte k notebooku `02_ZpracovaniDat.ipynb`, kde z těchto HTML souborů extrahujeme strukturovaná data.

---

## Tipy pro vylepšení kódu

1. **Ošetření chyb:** Přidejte try-except blok pro případ, že některá stránka není dostupná

```python
try:
    browser.get(url)
    # ...
except Exception as e:
    print(f"Chyba při stahování stránky {i}: {e}")
```

2. **Kontrola existence souborů:** Před stažením zkontrolujte, zda soubor již neexistuje

```python
nazev_souboru = f"data/raw/hockey_teams_page_{str(i).zfill(2)}.html"
if os.path.exists(nazev_souboru):
    print(f"Soubor {nazev_souboru} již existuje, přeskakuji...")
    continue
```

3. **Progress bar:** Pro delší stahování použijte knihovnu tqdm pro zobrazení průběhu

```python
from tqdm import tqdm

for i in tqdm(range(1, POCET_STRANEK + 1), desc="Stahování"):
    # ...
```

4. Pokud chcete spustit prohlížeč bez zobrazení okna (headless režim):
```python
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument('--headless')
browser = webdriver.Chrome(options=options)
```